## MVTEc    Visualizer

- claculates clip embeddings
- vizualisation via tensorboard projector

In [1]:
# activate clip env
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import numpy as np
import os
import clip
import torch
import pandas as pd

import warnings
warnings.filterwarnings("ignore", message=".*pandas only supports SQLAlchemy connectable.*")


torch.cuda.empty_cache()
print(f'GPU is available: {torch.cuda.is_available()}')
print("Torch version:", torch.__version__)


from PIL import Image

import torchvision
import torch.utils.tensorboard 
from torch.utils.tensorboard import SummaryWriter


GPU is available: True
Torch version: 2.0.1+cu117


2023-10-23 12:08:41.121543: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-23 12:08:41.121571: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-23 12:08:41.121600: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-23 12:08:41.128477: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-23 12:08:41.899060: W tensorflow/compiler/

In [2]:

def convert_images_for_tensorboard(image_paths):
    # Preprocess the images
    transform_display = torchvision.transforms.Compose([
        torchvision.transforms.Resize(64),  # Resize the images 
        torchvision.transforms.CenterCrop(64),  # Crop the images
        torchvision.transforms.ToTensor()  # Convert images to tensors        
    ])

    images = []  # Array to store the preprocessed images
    for path in image_paths:
        image = Image.open(path).convert('RGB')  # Open the image and convert to RGB
        # Apply the display transformation
        image_display = transform_display(image)
        image_display = image_display.unsqueeze(0) # Add a batch dimension to the image
        images.append(image_display)

    images = torch.cat(images, dim=0)  # Concatenate the images along the batch dimension

    return images

In [3]:
# PATHs
PROJECT_DATA_PATH='/home/bule/projects/Transformaly_FUAD/data/mvtec_anomaly_detection'
LOG_DIR= '/home/bule/projects/Transformaly_FUAD/tensorboard_logs'

In [4]:
# paths to images
image_paths = []
for root, dirs, files in os.walk(PROJECT_DATA_PATH):
    if 'ground_truth' not in root:
        for file in files:
            if file.endswith('.png'):
                image_paths.append(os.path.join(root, file))
len(image_paths)
## TODO embedd all images from MVTEC in CLIP space





5354

In [5]:
BATCH_SIZE = 1024  # Adjust this based on your GPU memory
FILENAME = 'MVTEC_embeddings_df'

if not os.path.exists(os.path.join(PROJECT_DATA_PATH,FILENAME+'.pkl')):

    # Load the model
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, transform = clip.load('ViT-B/32', device=device)


    # Function to get embeddings for a batch of images
    def get_batch_embeddings(batch_paths):
        images = [Image.open(p) for p in batch_paths]
        tensors = [transform(img) for img in images]
        batch_tensor = torch.stack(tensors).to(device)
        with torch.no_grad():
            batch_features = model.encode_image(batch_tensor)
        return batch_features.cpu().numpy()

    # Calculate embeddings for all images in batches
    all_embeddings = []
    for i in range(0, len(image_paths), BATCH_SIZE):
        batch_paths = image_paths[i:i+BATCH_SIZE]
        batch_embeddings = get_batch_embeddings(batch_paths)
        all_embeddings.extend(batch_embeddings)
        print(f"Processed images {i} to {i + len(batch_embeddings)}")

    # Convert all embeddings to DataFrame and save
    df = pd.DataFrame(all_embeddings, index=image_paths)
    df.to_pickle(os.path.join(PROJECT_DATA_PATH, FILENAME + '.pkl'))
else:
    df = pd.read_pickle(os.path.join(PROJECT_DATA_PATH,FILENAME+'.pkl'))

df.head()


,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
/home/bule/projects/Transformaly_FUAD/data/mvtec_anomaly_detection/pill/train/good/058.png,-0.255127,-0.003624,0.163086,0.293213,0.319824,0.009521,0.131104,0.075317,0.072937,0.185791,...,0.122559,0.422119,0.032654,-0.417236,-0.247314,0.374512,-0.308350,0.668945,0.321045,0.006065
/home/bule/projects/Transformaly_FUAD/data/mvtec_anomaly_detection/pill/train/good/065.png,-0.144531,-0.184204,0.286377,0.187256,0.088867,-0.159668,0.058960,0.182617,0.172485,0.022232,...,-0.032013,0.256592,-0.087402,-0.712402,-0.010551,0.552734,0.027878,0.615723,0.491943,-0.000437
/home/bule/projects/Transformaly_FUAD/data/mvtec_anomaly_detection/pill/train/good/173.png,-0.311279,-0.250977,0.210327,0.214722,0.283203,0.077942,-0.134033,0.241943,0.173828,-0.167847,...,0.159424,0.445801,0.003481,-0.574707,0.131104,0.278076,-0.070374,0.660156,0.447266,-0.256104
/home/bule/projects/Transformaly_FUAD/data/mvtec_anomaly_detection/pill/train/good/044.png,-0.166626,-0.156250,0.235596,0.190674,0.383789,0.034058,0.002312,0.236084,0.219971,-0.086060,...,0.129639,0.190063,0.038422,-0.464600,0.044800,0.020004,-0.088989,0.745117,0.257080,-0.310547
/home/bule/projects/Transformaly_FUAD/data/mvtec_anomaly_detection/pill/train/good/175.png,-0.186401,-0.187744,0.048248,0.418213,0.122986,0.156372,0.259521,0.132690,0.138916,-0.100769,...,0.267090,0.516602,0.001023,-0.426025,-0.120300,0.278564,-0.290771,0.668457,0.419434,-0.028442


In [6]:
# make tensorboard logs

print(FILENAME)
TENSORBOARD_LOGS_PATH=os.path.join(LOG_DIR,'_'+FILENAME).replace(" ", "_")


if not  os.path.exists(TENSORBOARD_LOGS_PATH):

    df = pd.read_pickle(os.path.join(PROJECT_DATA_PATH,FILENAME+'.pkl'))

    # Convert the resulting series of lists to a NumPy array
    numpy_array = df.to_numpy()

    # resize images 
    images = convert_images_for_tensorboard(image_paths)

    # for tensorboard
    writer = SummaryWriter(TENSORBOARD_LOGS_PATH)
    writer.add_embedding(numpy_array, label_img=images)

else:
    df = pd.read_pickle(os.path.join(PROJECT_DATA_PATH,FILENAME+'.pkl'))

print(f'{FILENAME}  number of images:{len(image_paths)}')

MVTEC_embeddings_df
MVTEC_embeddings_df  number of images:5354


### run tensorboard on port forward to browser

In [7]:
!tensorboard  --logdir $TENSORBOARD_LOGS_PATH

2023-10-23 12:09:07.193446: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-23 12:09:07.193486: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-23 12:09:07.193526: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-23 12:09:07.205167: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-23 12:09:08.241029: W tensorflow/compiler/